In [ ]:
# NOTES FROM MEETING
# No cell should take more than 20 minutes to run
# Please limit to one temporary collection (must be deleted at end of notebook). No external files
# Try to reduce single-line cells

# Mini-competition -
# Challenge: Limit to 50 lines of code (75 for team 1)! (excluding imports)
# Fit Success Rate: Fitting method with highest fit success rate
# Fit Average error: Fitting method with lowest fit error rate
# Fit Time complexity: Fitting method with lowest time expended
# Notebook housekeeping: Cleanest-looking notebook win!

# Sigmoid Criteria Curve Fitting: Algebraic Approach
**Contributors:** Justin Kaufman, Marco Scialanga

**Achievement:**

**Requirements:**


## Example of cell markdown
**This is a note:** Please have markdown before each cell. In this format!

In [ ]:
# Import packages you need
import sys
sys.path.append('../..')
import numpy as np
import matplotlib.pyplot as plt
from pymongo import MongoClient
import math
from scipy.optimize import curve_fit
from exploration.config import mongo_inst

In [ ]:
# Instantiate your mongo instance, classes, and any config/global vars
osu_db = mongo_inst['osu_random_db']

In [ ]:
# Load the curve data

In [ ]:
# Run all CDF curve fits & store in beatmap_criteria_curve collection

In [ ]:
# Load the beatmap attribs data

In [ ]:
# fit success-rate summary table with (3 rows 1 for each CDF fit) (4 columns 1* - 4*, 4* - 6*, 6* plus, all)

In [ ]:
# for curves that could be fit, error summary table with same layout as above

## Conclusion
Don't forget to provide a conclusion

In [ ]:
osu_db = mongo_inst['osu_random_db']

In [ ]:
love = osu_db['beatmap_criteria_curve'].find_one({'_id': 104229})

In [ ]:
x=np.arange(0,98)
y=np.asarray(love['no_mod']['n_pass'])/np.asarray(love['no_mod']['total'])
plt.plot(x,y)

In [ ]:
def logifunc(x,A,x0,k,off):
    return A / (1 + np.exp(-k*(x-x0)))+off

In [ ]:
y=y[np.logical_not(np.isnan(y))]
x=makeArr(getLen(y))

In [ ]:
popt, pcov = curve_fit(logifunc, x, y)
plt.plot(x,y,label='original')
plt.plot(x, logifunc(x, *popt), 'r-',label='Fitted logistic function')
plt.legend()

In [ ]:
popt, pcov = curve_fit(logifunc, x, y)
##func need to create a function that resembles the data, then pass that function in
plt.plot(x, logifunc(x, *popt), label="Fitted Curve")

In [ ]:
def makeArr(x):
    n=np.arange(0,x)
    return n

In [ ]:
makeArr(getLen(y))

In [ ]:
def fit_logistic(beatmap_id):
   
    beatmap = osu_db['beatmap_criteria_curve'].find_one({'_id': beatmap_id})
    
    def logifunc(x, A, x0, k, off):
        f = A / (1 + np.exp(-k * (x-x0))) + off
        return f

    def makeArr(x):
        n = np.arange(0,x)
        return n
    
    x_temp = np.arange(0,98)
    y_temp = np.asarray(beatmap['no_mod']['n_pass'])/np.asarray(beatmap['no_mod']['total'])
    y = y_temp[np.logical_not(np.isnan(y_temp))]
    x = makeArr(len(y))
   
    
    popt, pcov = curve_fit(logifunc, x, y)
    f, ax = plt.subplots(figsize = (14, 12))
    plt.title('Fitting logistic function for beatmap %d' %(beatmap_id))
    plt.plot(x,y,label = 'original')
    plt.plot(x, logifunc(x, *popt), 'r-',label = 'Fitted logistic function')
    plt.legend()
    
    return popt

In [ ]:
print(fit_logistic(104229))

In [ ]:
def fit_algebraic(beatmap_id):
   
    beatmap = osu_db['beatmap_criteria_curve'].find_one({'_id': beatmap_id})
    
    def func(x, A, x0 , k, off):
        f = A * (k*x - x0)/ (np.sqrt((k*x-x0)**2 + 1)) + off
        return f

    def makeArr(x):
        n = np.arange(0,x)
        return n
    
    x_temp = np.arange(0,98)
    y_temp = np.asarray(beatmap['no_mod']['n_pass'])/np.asarray(beatmap['no_mod']['total'])
    y = y_temp[np.logical_not(np.isnan(y_temp))]
    x = makeArr(len(y))
   
    return curve_fit(func, x, y)

In [ ]:
popt, pcov = fit_algebraic(104229)
f, ax = plt.subplots(figsize = (14, 12))
plt.title('Fitting algebraic function for beatmap %d' %(104229))
plt.plot(x,y,label = 'original')
plt.plot(x, func(x, *popt), 'r-',label = 'Fitted algebraic function')
plt.legend()

In [ ]:
def fit_generalized_logistic(beatmap_id):
   
    beatmap = osu_db['beatmap_criteria_curve'].find_one({'_id': beatmap_id})
    
    def func(x, A, x0 , k, off, a):
        f = A * (1 + np.exp(k*x - x0))**(-a)
        return f

    def makeArr(x):
        n = np.arange(0,x)
        return n
    
    x_temp = np.arange(0,98)
    y_temp = np.asarray(beatmap['no_mod']['n_pass'])/np.asarray(beatmap['no_mod']['total'])
    y = y_temp[np.logical_not(np.isnan(y_temp))]
    x = makeArr(len(y))
   
    param_bounds=([-np.inf,-np.inf, -np.inf,-np.inf,-np.inf, 0],[np.inf, np.inf,np.inf,np.inf,np.inf,np.inf])
    popt, pcov = curve_fit(func, x, y, maxfev = 50000)
    f, ax = plt.subplots(figsize = (14, 12))
    plt.title('Fitting generalized logistic function for beatmap %d' %(beatmap_id))
    plt.plot(x,y,label = 'original')
    plt.plot(x, func(x, *popt), 'r-',label = 'Fitted gen. logistic function')
    plt.legend()
    
    return True

In [ ]:
fit_generalized_logistic(104229)